In [1]:
import ee
import collections
collections.Callable = collections.abc.Callable
ee.Authenticate()
ee.Initialize()

Enter verification code:  4/1AX4XfWhSdvhT6oH3LQh3T4QKlRXuOyMdSZaWEA530eb6ViEk53w3TY5LxFE



Successfully saved authorization token.


#### next steps: add in other indices, specifically moisture

## Imports & Visualization Parameters

In [60]:
import ee
import geemap

geometry = ee.Geometry.Polygon(
        [[[25.67758947139908, -10.52500539578282],
          [25.67758947139908, -10.831343855184292],
          [26.285562854675604, -10.831343855184292],
          [26.285562854675604, -10.52500539578282]]], None, False)
mines = ee.FeatureCollection("users/EmilyNason/cod_mines_curated_all_opendata_p_ipis")
gsw = ee.Image("JRC/GSW1_3/GlobalSurfaceWater")
s2 = ee.ImageCollection("COPERNICUS/S2_SR")
s1 = ee.ImageCollection('COPERNICUS/S1_GRD')

rgbVis = {
  'bands': ['B4', 'B3', 'B2'],
  'min': 0,
  'max': 3000,
}

ndviVis = {
    'min': 0,
    'max': 1,
    'palette': ['white', 'green']
}

## True Color Composites

In [51]:
def create_median_composites(geometry, start, end):
    images = []
    
    for year in range(start, end+1):
        filtered = s2 \
            .filter(ee.Filter.lt('CLOUDY_PIXEL_PERCENTAGE', 30)) \
            .filter(ee.Filter.intersects('.geo', geometry)) \
            .filter(ee.Filter.date(str(year) + '-01-01', str(year) + '-12-31')) \

        composite = filtered.median().clip(geometry)
        
        images.append((composite, year))
        
    return images

def add_median_composites(images, geometry):
    Map = geemap.Map()
    Map.centerObject(geometry, 11)
    
    for image, year in images:
        Map.addLayer(image, rgbVis, str(year) + ' Median Composite')
        
    return Map

In [52]:
images = create_median_composites(geometry, 2019, 2021)
Map = add_median_composites(images, geometry)
Map

Map(center=[-10.67819783565169, 25.98157616303779], controls=(WidgetControl(options=['position', 'transparent_…

## NDVI

In [61]:
def create_ndvi_layers(images, geometry):
    Map = geemap.Map()
    Map.centerObject(geometry, 11)
    
    for image, year in images:
        ndvi = image.normalizedDifference(['B8', 'B4'])
        Map.addLayer(ndvi, ndviVis, 'NDVI ' + str(year))
        
    return Map

In [62]:
Map = create_ndvi_layers(images, geometry)
Map

Map(center=[-10.67819783565169, 25.98157616303779], controls=(WidgetControl(options=['position', 'transparent_…

## VH Band

In [64]:
def create_vh_composites(geometry, start, end):
    images = []
    
    for year in range(start, end+1):
        filtered = s1 \
            .filter(ee.Filter.eq('instrumentMode','IW')) \
            .filter(ee.Filter.listContains('transmitterReceiverPolarisation', 'VH')) \
            .filter(ee.Filter.eq('orbitProperties_pass', 'DESCENDING')) \
            .filter(ee.Filter.eq('resolution_meters',10)) \
            .filter(ee.Filter.intersects('.geo', geometry)) \
            .filter(ee.Filter.date(str(year) + '-01-01', str(year) + '-12-31')) \
            .select('VH')

        composite = filtered.mosaic().clip(geometry)
        
        images.append((composite, year))
        
    return images

def add_vh_composites(images, geometry):
    Map = geemap.Map()
    Map.centerObject(geometry, 11)
    
    for image, year in images:
        Map.addLayer(image, {'min':-25, 'max':0}, str(year) + ' VH Composite')
        
    return Map

In [65]:
imagesS1vh = create_vh_composites(geometry, 2019, 2021)
Map = add_vh_composites(imagesS1vh, geometry)
Map

Map(center=[-10.67819783565169, 25.98157616303779], controls=(WidgetControl(options=['position', 'transparent_…

### VH Calculations

In [66]:
cpThreshold = 8
permanentWater = gsw.select('seasonality').gte(7).clip(geometry)

In [67]:
def create_vh_calcs(images, geometry):
    Map = geemap.Map()
    Map.centerObject(geometry, 11)
    
    for image, year in images:
        waterStorage = image.lt(-22).rename('water').selfMask()
        connectW = waterStorage.connectedPixelCount(25)
        waterStorage = waterStorage.updateMask(connectW.gt(cpThreshold))
        Map.addLayer(waterStorage, {'min':0, 'max':1, 'palette': ['blue']}, 'Water ' + str(year))
        
        filtered = s2 \
            .filter(ee.Filter.lt('CLOUDY_PIXEL_PERCENTAGE', 30)) \
            .filter(ee.Filter.intersects('.geo', geometry)) \
            .filter(ee.Filter.date(str(year) + '-01-01', str(year) + '-12-31')) \

        composite = filtered.median().clip(geometry)
        
        ndvi = composite.normalizedDifference(['B8', 'B4'])
        
        mining = image.gte(-22).And(image.lt(-18)).And(ndvi.lt(.1)).rename('mine').selfMask()
        mining = mining.where(permanentWater,0).selfMask()
        connectM = mining.connectedPixelCount(25)
        mining = mining.updateMask(connectM.gt(cpThreshold))
        Map.addLayer(mining, {'min':0, 'max':1, 'palette': ['red']}, 'Mining Estimate ' + str(year))
        
    return Map

In [68]:
Map = create_vh_calcs(imagesS1vh, geometry)
legend_dict = {
    'Water': ('0000FF'), # blue
    'Mine Estimate': ('FF0000'), # red
}
Map.add_legend(legend_title='Legend', legend_dict=legend_dict, position='bottomright')

Map

Map(center=[-10.67819783565169, 25.98157616303779], controls=(WidgetControl(options=['position', 'transparent_…

## VV Band

In [69]:
def create_vv_composites(geometry, start, end):
    images = []
    
    for year in range(start, end+1):
        filtered = s1 \
            .filter(ee.Filter.eq('instrumentMode','IW')) \
            .filter(ee.Filter.listContains('transmitterReceiverPolarisation', 'VV')) \
            .filter(ee.Filter.eq('orbitProperties_pass', 'DESCENDING')) \
            .filter(ee.Filter.eq('resolution_meters',10)) \
            .filter(ee.Filter.intersects('.geo', geometry)) \
            .filter(ee.Filter.date(str(year) + '-01-01', str(year) + '-12-31')) \
            .select('VV')

        composite = filtered.mosaic().clip(geometry)
        
        images.append((composite, year))
        
    return images

def add_vv_composites(images, geometry):
    Map = geemap.Map()
    Map.centerObject(geometry, 11)
    
    for image, year in images:
        Map.addLayer(image, {'min':-25, 'max':0}, str(year) + ' VV Composite')
        
    return Map

In [70]:
imagesS1vv = create_vv_composites(geometry, 2019, 2021)
Map = add_vv_composites(imagesS1vv, geometry)
Map

Map(center=[-10.67819783565169, 25.98157616303779], controls=(WidgetControl(options=['position', 'transparent_…

### VV Calculations

In [71]:
def create_vv_calcs(images, geometry):
    Map = geemap.Map()
    Map.centerObject(geometry, 11)
    
    for image, year in images:
        waterStorage = image.lt(-15).rename('water').selfMask()
        connectW = waterStorage.connectedPixelCount(25)
        waterStorage = waterStorage.updateMask(connectW.gt(cpThreshold))
        Map.addLayer(waterStorage, {'min':0, 'max':1, 'palette': ['blue']}, 'Water ' + str(year))
        
        filtered = s2 \
            .filter(ee.Filter.lt('CLOUDY_PIXEL_PERCENTAGE', 30)) \
            .filter(ee.Filter.intersects('.geo', geometry)) \
            .filter(ee.Filter.date(str(year) + '-01-01', str(year) + '-12-31')) \

        composite = filtered.median().clip(geometry)
        
        ndvi = composite.normalizedDifference(['B8', 'B4'])
        
        mining = image.gte(-15).And(image.lt(-9)).And(ndvi.lt(.1)).rename('mine').selfMask()
        mining = mining.where(permanentWater,0).selfMask()
        connectM = mining.connectedPixelCount(25)
        mining = mining.updateMask(connectM.gt(cpThreshold))
        Map.addLayer(mining, {'min':0, 'max':1, 'palette': ['red']}, 'Mining Estimate ' + str(year))
        
    return Map

In [74]:
Map = create_vv_calcs(imagesS1vv, geometry)
legend_dict = {
    'Water': ('0000FF'), # blue
    'Mine Estimate': ('FF0000'), # red
}
Map.add_legend(legend_title='Legend', legend_dict=legend_dict, position='bottomright')

Map

Map(center=[-10.67819783565169, 25.98157616303779], controls=(WidgetControl(options=['position', 'transparent_…

## VV/VH Ratio

In [78]:
def create_vv_vh_composites(geometry, start, end):
    images = []
    
    for year in range(start, end+1):
        filtered = s1 \
            .filter(ee.Filter.eq('instrumentMode','IW')) \
            .filter(ee.Filter.eq('orbitProperties_pass', 'DESCENDING')) \
            .filter(ee.Filter.eq('resolution_meters',10)) \
            .filter(ee.Filter.intersects('.geo', geometry)) \
            .filter(ee.Filter.date(str(year) + '-01-01', str(year) + '-12-31')) \
            .select(['VV', 'VH'])

        composite = filtered.mosaic().clip(geometry)
        
        images.append((composite, year))
        
    return images

def add_vv_vh_composites(images, geometry):
    Map = geemap.Map()
    Map.centerObject(geometry, 11)
    
    for image, year in images:
        Map.addLayer(image.select('VV').divide(image.select('VH')), {}, str(year) + ' VV/VH')
        
    return Map

In [79]:
imagesS1ratio = create_vv_vh_composites(geometry, 2019, 2021)
Map = add_vv_vh_composites(imagesS1ratio, geometry)
Map

Map(center=[-10.67819783565169, 25.98157616303779], controls=(WidgetControl(options=['position', 'transparent_…

### VV/VH Calculations

In [100]:
def create_vv_vh_calcs(images, geometry):
    Map = geemap.Map()
    Map.centerObject(geometry, 11)
    
    for image, year in images:
        # not picking up the lake in 2019 -> why
        waterStorage = image.select('VV').divide(image.select('VH')).gt(0.6).And(image.select('VH').lt(-22)).rename('water').selfMask()
        connectW = waterStorage.connectedPixelCount(25)
        waterStorage = waterStorage.updateMask(connectW.gt(cpThreshold))
        Map.addLayer(waterStorage, {'min':0, 'max':1, 'palette': ['blue']}, 'Water ' + str(year))
        
        filtered = s2 \
            .filter(ee.Filter.lt('CLOUDY_PIXEL_PERCENTAGE', 30)) \
            .filter(ee.Filter.intersects('.geo', geometry)) \
            .filter(ee.Filter.date(str(year) + '-01-01', str(year) + '-12-31')) \

        composite = filtered.median().clip(geometry)
        
        ndvi = composite.normalizedDifference(['B8', 'B4'])
        
        #currently not doing much -> basically going off of NDVI
        mining = image.select('VV').divide(image.select('VH')).lt(0.6).And(ndvi.lt(.1)).rename('mine').selfMask()
        mining = mining.where(permanentWater,0).selfMask()
        connectM = mining.connectedPixelCount(25)
        mining = mining.updateMask(connectM.gt(cpThreshold))
        Map.addLayer(mining, {'min':0, 'max':1, 'palette': ['red']}, 'Mining Estimate ' + str(year))
        
    return Map

In [101]:
Map = create_vv_vh_calcs(imagesS1ratio, geometry)

legend_dict = {
    'Water': ('0000FF'), # blue
    'Mine Estimate': ('FF0000'), # red
}
Map.add_legend(legend_title='Legend', legend_dict=legend_dict, position='bottomright')

Map

Map(center=[-10.67819783565169, 25.98157616303779], controls=(WidgetControl(options=['position', 'transparent_…